# Requirments



In [ ]:
"""
# get majka database
!curl --remote-name-all https://nlp.fi.muni.cz/ma{/majka.w-lt}
!mv majka.w-lt drive/MyDrive/data/
# download czech squad
!curl --remote-name-all https://lindat.cz/repository/xmlui/bitstream/handle/11234/1-3069{/sqad_v3.tar.xz}
!mv sqad_v3.tar.xz drive/MyDrive/data/
!tar -xf drive/MyDrive/data/sqad_v3.tar.xz
"""

In [ ]:
!pip install sentencepiece
!pip install datasets transformers
!pip install googletrans==4.0.0-rc1
!pip install wikipedia
!pip install rank_bm25
!pip install majka

In [2]:
import torch
import string
import os
import sys
import time
import shutil
import json
import numpy as np
import collections
import datetime
from tqdm.auto import tqdm

from datasets import load_dataset, load_metric
from typing import List, Tuple, Dict
from collections import defaultdict
from transformers import AlbertTokenizerFast, AlbertForQuestionAnswering, TrainingArguments, Trainer, default_data_collator

from rank_bm25 import BM25Okapi
import re
import majka
import wikipedia
from googletrans import Translator

from google.colab import drive

In [3]:
# Remove pre-cached sample data in colab's directory
if os.path.isdir("sample_data"):
  shutil.rmtree("sample_data")
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
def get_timestamp():
    return datetime.datetime.now().strftime('%Y-%m-%d_%H:%M')

Print gpu

In [144]:
print("Current working directory: " + os.getcwd())
print(f"python version: {sys.version}")
print(f"torch version: {torch.__version__}")
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    !nvidia-smi

Current working directory: /content
python version: 3.7.10 (default, Feb 20 2021, 21:17:23) 
[GCC 7.5.0]
torch version: 1.7.1+cu101
Tesla T4
Mon Mar  1 21:12:56 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   71C    P0    31W /  70W |   2860MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+--------------------------

In [112]:
# This flag is the difference between SQUAD v1 or 2 (if you're using another dataset, it indicates if impossible
# answers are allowed or not).
squad_v2 = True
if squad_v2:
  model_checkpoint = "./drive/MyDrive/albert_models/albert_squad2_finetuned"
else:
  model_checkpoint = "./drive/MyDrive/albert_models/albert_finetuned"
batch_size = 16

# Data preprocessing

In [9]:
def prepare_train_features(examples):
    # Tokenize our examples with truncation and padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question"],
        examples["context"],
        truncation="only_second",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # The offset mappings will give us a map from token to character position in the original context. This will
    # help us compute the start_positions and end_positions.
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Let's label those examples!
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        # If no answers are given, set the cls_index as answer.
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != 1:
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != 1:
                token_end_index -= 1

            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

In [33]:
tokenizer = AlbertTokenizerFast.from_pretrained(model_checkpoint)
datasets = load_dataset("squad_v2" if squad_v2 else "squad")

Dataset squad_v2 downloaded and prepared to /root/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/9cac55034b086140f0649ecb5c604d09d7da2f2f5b73a90caa2e2bcc1f5cac09. Subsequent calls will reuse this data.


In [34]:
max_length=384
doc_stride=128
tokenized_datasets = datasets.map(prepare_train_features,
                                  batched=True, 
                                  remove_columns=datasets["train"].column_names)

# Model fine-tuning

In [37]:
model = AlbertForQuestionAnswering.from_pretrained(model_checkpoint)

Create trainer

In [38]:
args = TrainingArguments(
    f"./drive/MyDrive/data/checkpoints",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
)

data_collator = default_data_collator

trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

Train and save

In [ ]:
trainer.train()

In [ ]:
trainer.save_model("./drive/MyDrive/data/albert_finetuned")

# Model evaluation

In [16]:
def prepare_validation_features(examples):
    # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question"],
        examples["context"],
        truncation="only_second",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    # We keep the example_id that gave us this feature and we will store the offset mappings.
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1
        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])

        # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token
        # position is part of the context or not.
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples

In [27]:
def postprocess_qa_predictions(examples, features, raw_predictions, n_best_size = 20, max_answer_length = 30):
    all_start_logits, all_end_logits = raw_predictions
    # Build a map example to its corresponding features.
    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)

    # The dictionaries we have to fill.
    predictions = collections.OrderedDict()

    # Logging.
    print(f"Post-processing {len(examples)} example predictions split into {len(features)} features.")

    # Let's loop over all the examples!
    for example_index, example in enumerate(tqdm(examples)):
        # Those are the indices of the features associated to the current example.
        feature_indices = features_per_example[example_index]

        min_null_score = None # Only used if squad_v2 is True.
        valid_answers = []
        
        context = example["context"]
        # Looping through all the features associated to the current example.
        for feature_index in feature_indices:
            # We grab the predictions of the model for this feature.
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
            # This is what will allow us to map some the positions in our logits to span of texts in the original
            # context.
            offset_mapping = features[feature_index]["offset_mapping"]

            # Update minimum null prediction.
            cls_index = features[feature_index]["input_ids"].index(tokenizer.cls_token_id)
            feature_null_score = start_logits[cls_index] + end_logits[cls_index]
            if min_null_score is None or min_null_score < feature_null_score:
                min_null_score = feature_null_score

            # Go through all possibilities for the `n_best_size` greater start and end logits.
            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
                    # to part of the input_ids that are not in the context.
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or offset_mapping[end_index] is None
                    ):
                        continue
                    # Don't consider answers with a length that is either < 0 or > max_answer_length.
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue

                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]
                    valid_answers.append(
                        {
                            "score": start_logits[start_index] + end_logits[end_index],
                            "text": context[start_char: end_char]
                        }
                    )
        
        if len(valid_answers) > 0:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
        else:
            # In the very rare edge case we have not a single non-null prediction, we create a fake prediction to avoid
            # failure.
            best_answer = {"text": "", "score": 0.0}
        
        # Let's pick our final answer: the best one or the null answer (only for squad_v2)
        if not squad_v2:
            predictions[example["id"]] = best_answer["text"]
        else:
            answer = best_answer["text"] if best_answer["score"] > min_null_score else ""
            predictions[example["id"]] = answer

    return predictions

Final evaluation

In [39]:
# get ground truth features
validation_features = datasets["validation"].map(prepare_validation_features,
                                                 batched=True,
                                                 remove_columns=datasets["validation"].column_names)

# get raw predictions
raw_predictions = trainer.predict(validation_features)

validation_features.set_format(type=validation_features.format["type"], 
                               columns=list(validation_features.features.keys()))

In [41]:
# hyperparameters
max_answer_length = 30
n_best_size = 20

# map examples to features
examples = datasets["validation"]
features = validation_features

example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
features_per_example = collections.defaultdict(list)
for i, feature in enumerate(features):
    features_per_example[example_id_to_index[feature["example_id"]]].append(i)

# get final predictions
final_predictions = postprocess_qa_predictions(datasets["validation"], validation_features, raw_predictions.predictions)
# get metric used
metric = load_metric("squad_v2" if squad_v2 else "squad")

#evaluate
if squad_v2:
    formatted_predictions = [{"id": k, "prediction_text": v, "no_answer_probability": 0.0} for k, v in final_predictions.items()]
else:
    formatted_predictions = [{"id": k, "prediction_text": v} for k, v in final_predictions.items()]
references = [{"id": ex["id"], "answers": ex["answers"]} for ex in datasets["validation"]]
metric.compute(predictions=formatted_predictions, references=references)

Post-processing 11873 example predictions split into 12171 features.


{'HasAns_exact': 74.2914979757085,
 'HasAns_f1': 80.7472515516722,
 'HasAns_total': 5928,
 'NoAns_exact': 82.35492010092514,
 'NoAns_f1': 82.35492010092514,
 'NoAns_total': 5945,
 'best_exact': 78.3289817232376,
 'best_exact_thresh': 0.0,
 'best_f1': 81.55223677236683,
 'best_f1_thresh': 0.0,
 'exact': 78.3289817232376,
 'f1': 81.55223677236697,
 'total': 11873}

# Decode

In [171]:
def decode(output, context, offset_mapping):
  start_logits = output.start_logits[0].cpu().detach().numpy()
  end_logits = output.end_logits[0].cpu().detach().numpy()
  offset_mapping = offset_mapping[0]

  # Gather the indices the best start/end logits:
  start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
  end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
  valid_answers = []
  for start_index in start_indexes:
      for end_index in end_indexes:
          # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
          # to part of the input_ids that are not in the context.
          if (
              start_index >= len(offset_mapping)
              or end_index >= len(offset_mapping)
              or offset_mapping[start_index] is None
              or offset_mapping[end_index] is None
          ):
              continue
          # Don't consider answers with a length that is either < 0 or > max_answer_length.
          if end_index < start_index or end_index - start_index + 1 > max_answer_length:
              continue
          if start_index <= end_index: # We need to refine that test to check the answer is inside the context
              start_char = offset_mapping[start_index][0]
              end_char = offset_mapping[end_index][1]
              valid_answers.append(
                  {
                      "score": start_logits[start_index] + end_logits[end_index],
                      "text": context[start_char: end_char].strip()
                  }
              )

  valid_answers = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[:n_best_size]
  return valid_answers

Get answers

In [174]:
def get_answers(question, context):
  inputs = tokenizer(question, context, 
                     return_tensors='pt',
                     truncation="only_second",
                     max_length=max_length,
                     stride=doc_stride,
                     return_offsets_mapping=True,
                     padding="max_length")
  inputs.to(device)

  outputs = model(inputs['input_ids'], 
                  token_type_ids=inputs['token_type_ids'],
                  attention_mask=inputs['attention_mask'])
  
  valid_answers = decode(outputs, context, inputs['offset_mapping'])
  return valid_answers # [0]['text']

# Loading model


Choose trained version

In [136]:
squad_v2 = False
if squad_v2:
  model_checkpoint = "./drive/MyDrive/albert_models/albert_squad2_finetuned"
else:
  model_checkpoint = "./drive/MyDrive/albert_models/albert_finetuned"

Load model and tokenizer

In [146]:
tokenizer = AlbertTokenizerFast.from_pretrained(model_checkpoint)
model = AlbertForQuestionAnswering.from_pretrained(model_checkpoint).to(device)

In [128]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Model has {count_parameters(model)} parameters")

Model has 11094530 parameters


Asking questions

In [179]:
# give question and context
question = "For how much was Chicago television station sold?"
context = "In 1952, when the release of the FCC's Sixth Report and Order announced the end of its freeze" \
          " on new station license applications, among the issues the Commission was slated to address was" \
          " whether to approve the UPT-ABC merger. One FCC Commissioner saw the possibility of ABC, funded" \
          " by UPT, becoming a viable and competitive third television network. On February 9, 1953, the FCC" \
          " approved UPT's purchase of ABC in exchange for $25 million in shares. The merged company, renamed" \
          " American Broadcasting-Paramount Theatres, Inc. and headquartered in the Paramount Building at 1501 " \
          "Broadway in Manhattan, owned six AM and several FM radio stations, five television stations and 644 " \
          "cinemas in 300 U.S. cities. To comply with FCC ownership restrictions in effect at the time that" \
          " barred common ownership of two television stations in the same market, UPT sold its Chicago " \
          "television station, WBKB-TV, to CBS (which subsequently changed the station's call letters to WBBM-TV) " \
          "for $6 million, while it kept ABC's existing Chicago station, WENR-TV. The merged company acquired the WBKB " \
          "call letters for channel 7, which would eventually become WLS-TV. Goldenson began to sell some of the older " \
          "theaters to help finance the new television network."

answers = get_answers(question, context)
print(answers[0])
print(answers[0]['text'])

{'score': 23.636002, 'text': '$6 million'}
$6 million


# Retriever


In [ ]:
translator = Translator()
wikipedia.set_lang("cs") 

In [ ]:
def extract_que_ans(dirnum, filename, lemmatized=False):
  # get question and answer from czech squad 
  f = open(f"drive/MyDrive/data/cz_sqad/data/{dirnum}/{filename}", "r")
  q = f.read().split("\n")
  question = ""

  #lemmatized parsed
  if lemmatized:
    answer = ""
    for line in q:
      if len(line.split()) < 2:
        continue
      line = line.split("\t")[1]
      if line in {"<s>", "<g/>", "</s>"}:
        answer = answer[:-1]
        continue
      if line != "?":
        answer += line + " "
      else:
        answer += " ? "

    f.close()
    return answer

  # normal parse
  for line in q:
    line = line.split("\t")[0]
    if line in {"<s>", "<g/>", "</s>"}:
      question = question[:-1]
      continue
    
    if line != "?":
      question += line + " "
    else:
      question += " ? "

  f.close()
  return question

In [ ]:
# save the most common czech words
common = "být a se v na ten on že s z který mít do já o k i jeho ale svůj jako za moci rok pro tak po tento co když všechen už jak aby od nebo říci jeden jen můj jenž člověk ty stát u muset velký chtít také až než ještě při jít pak před dva však ani vědět nový hodně podle další celý jiný mezi dát tady den tam kde doba každý místo dobrý takový strana protože nic začít něco vidět říkat ne sám bez či dostat nějaký proto"
common = common.split()
punctuation = ". , ? ! ... \" ( ) ; - /"
punctuation = punctuation.split()

# decides if query token is common
def iscommon(x):
  if x in common or x in punctuation:
    return True
  else:
    return False

# remove the most common czech words from the query tokens (low information value)
def delete_common(tokens):
  tokens = [x for x in tokens if not iscommon(x)]
      
  return tokens

In [ ]:
def search_again(tokens):

  searched_term = (' ').join(tokens)
  print(searched_term)
  doc_list = wikipedia.search(searched_term)

  if len(tokens) == 0:
    return []

  if len(doc_list) == 0:
    del tokens[0]
    return search_again(tokens)

  return doc_list

In [ ]:
morph = majka.Majka('drive/MyDrive/data/majka.w-lt')
morph.flags |= majka.ADD_DIACRITICS  # find word forms with diacritics
morph.flags |= majka.DISALLOW_LOWERCASE  # do not enable to find lowercase variants
morph.flags |= majka.IGNORE_CASE  # ignore the word case whatsoever
morph.flags = 0  # unset all flags

morph.tags = False  # return just the lemma, do not process the tags
morph.first_only = True  # return only the first entry
morph.negative = "ne"

# returns lemma of each token in a list of lemmatized tokens
def lemmatize(text):

  tok_text = text.lower()
  tok_text = re.split("\W", text)

  # lemmatize each token
  lemmatized_tokens = []
  for token in tok_text:
    if token == '':
      continue
    lemma = morph.find(token)
    if len(lemma) == 0:
      lemmatized_tokens.append(token)
    else:
      lemmatized_tokens.append(lemma[0]['lemma'])

  return lemmatized_tokens


In [ ]:
def retrieve(question):  

  #search for documents
  doc_list = wikipedia.search(question)

  # simplify the search if its too bad
  if len(doc_list) == 0:
    # extract important for wiki
    tokens = lemmatize(question)
    tokens = delete_common(tokens)
    doc_list = search_again(tokens)
    
  if len(doc_list) == 0:
      return ""

  # split docs into paragraphs
  pars = []
  max_docs = 3
  num_docs = 0

  for doc in doc_list:
    # get whole page content
    try:
      doc = wikipedia.page(doc)
    except wikipedia.DisambiguationError as e:
      s = e.options[0]
      doc = wikipedia.page(s)
    result = re.split('== .*. ==|\n\n', doc.content)

    # save stripped paragraphs
    for par in result:
      par = par.strip()
      par = par.strip('=')
      par = par.strip('\n')
      par = par.strip('\r\n')

      if par == '' or par == '\n':
        continue
      pars.append(par)

    num_docs += 1
    if num_docs >= max_docs:
      break

  # tokenize for bm25
  tok_text = []
  for par in pars:
    tok_par = par.lower()
    tok_par = re.split("\W", tok_par)
    for tok in tok_par:
      if tok == "":
        tok_par.remove("")
    tok_text.append(tok_par)

  # build index
  bm25 = BM25Okapi(tok_text)

  # tokenize the query
  tokenized_query = question.lower()
  tokenized_query = re.split("\W", tokenized_query)

  # get results
  results = bm25.get_top_n(tokenized_query, pars, n=3)

  return results


In [ ]:
# create very simple batch object
class simple_batch():
  def __init__(self,q,d,raw_d,d_pos):
    map_to_gpu_tensor = lambda x: torch.Tensor(x).long().to(torch.device("cuda")).unsqueeze(0)
    self.question=map_to_gpu_tensor(q)
    self.document=map_to_gpu_tensor(d)
    self.raw_document_context = [raw_d]
    self.document_token_positions = [d_pos]

# Testing with sqad

In [ ]:
# write results to
f = open("drive/MyDrive/data/saved_answers/test.txt", "w")

# write first question-answer pairs in sqad
from_q = 1
to_q = 1
for i in range(from_q, to_q+1):

  # get question number
  name = ""
  for _ in range(len(str(i)), 6):
    name += "0"
  name += str(i)

  # extract from dataset
  question = extract_que_ans(name, "01question.vert")
  correct_answer = extract_que_ans(name, "09answer_extraction.vert")
  lemmatized_answer = extract_que_ans(name, "09answer_extraction.vert", lemmatized=True)

  # wiki search
  documents = retrieve(question)
  # for saving the best results
  bestAnswers = []
  bestProbs = []
  bestDocs = []
  bestLogProbs = []

  question_cs = question # save czech question

  # iterate over retrieved paragraphs
  for document in documents:

    # strip whitespaces
    document = document.strip()

    # chceck if any document has been found for the question
    if document == "":
      f.write("question: " + question + "\n" +
              "answer: odpověď nenalezena" + "\n" + 
              "correct answer: " + correct_answer + "\n\n")
      continue;
    try:
      document_cs = document
      document = translator.translate(document, dest='en').text
    except TypeError:
      continue

    # remove some trash
    # TODO this not cool (in retriever)
    if (document_cs.strip().startswith("Obrázky, zvuky či videa k tématu")):
      continue


    bestDocs.append(document_cs)
    # translate
    question = translator.translate(question, dest='en').text

    # make sure the current vocab is model's vocab
    vocab = model.embedder.vocab

    # tokenization
    document_tokens, tokenized_document_list = tokenize(document)
    tokenized_question_list = tokenize(question)[1]

    # keep positions of each token in document, we will need this later, when decoding model outputs
    document_token_positions = [[token.idx, token.idx + len(token.text)] for token in document_tokens]

    # lowercasing and numericalization
    numericalized_document = [vocab.stoi[s.lower()] for s in tokenized_document_list]
    numericalized_question = [vocab.stoi[s.lower()] for s in tokenized_question_list]

    batch = simple_batch(numericalized_question,numericalized_document,document,document_token_positions)

    # get predictions with arg_maxes
    logprobs_S, logprobs_E, argmax_Q = model.forward(batch, return_max=True)

    # decode from log probabilities to predictions
    best_span_prob, candidate, logprobs = decode(logprobs_S, logprobs_E)
    confidence =  best_span_prob.item()
    log_conf = logprobs.item()

    #get answer
    answers = get_spans(batch, candidate)
    answer = answers[0]

    # save probs and answer
    bestAnswers.append(answer)
    bestProbs.append(confidence)
    bestLogProbs.append(log_conf)

  # check if any answer was found
  if len(bestProbs) == 0:
    f.write("question: " + question_cs + "\n" +
              "answer: odpověď nenalezena" + "\n" + 
              "correct answer: " + correct_answer + "\n\n")
    continue

  print(bestAnswers)
  print(bestProbs)
  print(bestLogProbs)
  # get the best doc
  # get best answer from retriever according to reader
  # get the best confidence
  document = bestDocs[np.argmax(bestLogProbs, axis=0)]
  answer = bestAnswers[np.argmax(bestLogProbs, axis=0)]
  confidence = bestProbs[np.argmax(bestLogProbs, axis=0)]

  # translate the final answer
  answer =  translator.translate(answer, dest='cs').text

  # write the result to file
  f.write("otázka č." + name + ": " + question_cs + "\n" +
          "::odpověď: " + answer + "\n" + 
          "::správná odpověď podle sqad : " + correct_answer + "\n" +
          # "::sqad lemmatized : " + lemmatized_answer + "\n" +
          "----------------------------------------------------------------\n"+
          "získaný dokument: " + document + 
          "\n----------------------------------------------------------------\n"+
          "----------------------------------------------------------------"+
          "\n\n")
  
  # controlprint
  print("wrote: " + name)

  # print(f"The answer is: \"{answer}\".")
  # print(f"The model is confident with {confidence:.2f} probability.")
f.close()